[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/weaviate-features/model-providers/databricks/hybrid_search_databricks.ipynb)

# Hybrid Search with Databricks

This recipe will show you how to run hybrid search with [embeddings from Databricks](https://weaviate.io/developers/weaviate/model-providers/databricks/embeddings).

## Requirements

1. Weaviate cluster
    1. You can create a 14-day free sandbox on [WCD](https://console.weaviate.cloud/)
    2. [Embedded Weaviate](https://weaviate.io/developers/weaviate/installation/embedded)
    3. [Local deployment](https://weaviate.io/developers/weaviate/installation/docker-compose#starter-docker-compose-file)
    4. [Other options](https://weaviate.io/developers/weaviate/installation)

2. Databricks token

3. Endpoint to an embedding model on Databricks

## Import Dependencies, Libraries, and Keys

In [ ]:
!pip install --q weaviate-client

In [1]:
import weaviate
from weaviate.classes.init import Auth
import weaviate.classes.config as wc
from weaviate.embedded import EmbeddedOptions
import weaviate.classes.query as wq


import os
import requests
import json

## Connect to Weaviate

Only choose one option from the below.

**Weaviate Cloud Deployment**

In [13]:
WCD_URL = os.environ["WEAVIATE_URL"] # Replace with your Weaviate cluster URL
WCD_AUTH_KEY = os.environ["WEAVIATE_AUTH"] # Replace with your cluster auth key
DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token


# Weaviate Cloud Deployment
client = weaviate.connect_to_wcs(
    cluster_url=WCD_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WCD_AUTH_KEY),
      headers={"X-Databricks-Token": DATABRICKS_TOKEN}
)

print(client.is_ready())

True


**Embedded Weaviate**

In [ ]:
# DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token

# client = weaviate.WeaviateClient(
#     embedded_options=EmbeddedOptions(
#         version="1.30.0",
#         additional_env_vars={
#             "ENABLE_MODULES": "text2vec-databricks"
#         }),
#         additional_headers={
#             "X-Databricks-Token": DATABRICKS_TOKEN
#         }
# )

# client.connect()

**Local Deployment**

In [ ]:
# DATABRICKS_TOKEN = os.environ["DATABRICKS_TOKEN"] # Replace with your Databricks token

# client = weaviate.connect_to_local(
#   headers={
#     "X-Databricks-Token": DATABRICKS_TOKEN
#   }
# )
# print(client.is_ready())

## Create a collection
> Collection stores your data and vector embeddings.

In [3]:
# Note: This will delete your data stored in "JeopardyQuestion" and
# it will require you to re-import again.

EMBEDDING_ENDPOINT = os.environ["EMBEDDING_ENDPOINT"] # Replace with the endpoint to your embedding model

# Delete the collection if it already exists
if (client.collections.exists("JeopardyQuestion")):
    client.collections.delete("JeopardyQuestion")

client.collections.create(
    name="JeopardyQuestion",

    vectorizer_config=wc.Configure.Vectorizer.text2vec_databricks(
        endpoint=EMBEDDING_ENDPOINT, # endpoint to the embedding model
    ),

    properties=[ # defining properties (data schema) is optional
        wc.Property(name="Question", data_type=wc.DataType.TEXT), 
        wc.Property(name="Answer", data_type=wc.DataType.TEXT),
        wc.Property(name="Category", data_type=wc.DataType.TEXT) 
    ]
)

print("Successfully created collection: JeopardyQuestion.")

Successfully created collection: JeopardyQuestion.


{"build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"warning","msg":"prop len tracker file /Users/erikacardenas/.local/share/weaviate/jeopardyquestion/UNvR5jmmimUt/proplengths does not exist, creating new tracker","time":"2025-05-02T15:09:01-04:00"}
{"action":"hnsw_prefill_cache_async","build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"info","msg":"not waiting for vector cache prefill, running in background","time":"2025-05-02T15:09:01-04:00","wait_for_cache_prefill":false}
{"build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version":"1.30.0","level":"info","msg":"Created shard jeopardyquestion_UNvR5jmmimUt in 10.411875ms","time":"2025-05-02T15:09:01-04:00"}
{"action":"hnsw_vector_cache_prefill","build_git_commit":"b7b77150ac","build_go_version":"go1.24.2","build_image_tag":"HEAD","build_wv_version

## Import the Data

In [4]:
url = 'https://raw.githubusercontent.com/weaviate/weaviate-examples/main/jeopardy_small_dataset/jeopardy_tiny.json'
resp = requests.get(url)
data = json.loads(resp.text)

# Get a collection object for "JeopardyQuestion"
jeopardy = client.collections.get("JeopardyQuestion")

# Insert data objects
response = jeopardy.data.insert_many(data)

# Note, the `data` array contains 10 objects, which is great to call insert_many with.
# However, if you have a milion objects to insert, then you should spit them into smaller batches (i.e. 100-1000 per insert)

if (response.has_errors):
    print(response.errors)
else:
    print("Insert complete.")

Insert complete.


## Hybrid Search

The `alpha` parameter determines the weight given to the sparse and dense search methods. `alpha = 0` is pure sparse (bm25) search, whereas `alpha = 1` is pure dense (vector) search. 

Alpha is an optional parameter. The default is set to `0.75`.

### Hybrid Search only

The below query is finding Jeopardy questions about animals and is limiting the output to only two results. Notice `alpha` is set to `0.80`, which means it is weighing the vector search results more than bm25. If you were to set `alpha = 0.25`, you would get different results. 

In [5]:
response = jeopardy.query.hybrid(
    query="northern beast",
    query_properties=["question"],
    alpha=0.8,
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: 2a35dc97-40c1-4ab1-bd4c-baaa63fd3b06
Data: {
  "answer": "species",
  "question": "2000 news: the Gunnison sage grouse isn't just another northern sage grouse, but a new one of this classification",
  "category": "SCIENCE"
} 

ID: e5aca3f1-1f00-435c-9d9b-c30ada64d128
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 

ID: 392fbc4c-fecf-4ffe-873c-d87e91f66e4a
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 



### Hybrid Search with a `where` filter

Find Jeopardy questions about elephants, where the category is set to Animals.

In [6]:
response = jeopardy.query.hybrid(
    query="northern beast",
    alpha=0.8,
    filters=wq.Filter.by_property("category").equal("Animals"),
    limit=3
)

for item in response.objects:
    print("ID:", item.uuid)
    print("Data:", json.dumps(item.properties, indent=2), "\n")

ID: e5aca3f1-1f00-435c-9d9b-c30ada64d128
Data: {
  "answer": "the diamondback rattler",
  "question": "Heaviest of all poisonous snakes is this North American rattlesnake",
  "category": "ANIMALS"
} 

ID: 392fbc4c-fecf-4ffe-873c-d87e91f66e4a
Data: {
  "answer": "Antelope",
  "question": "Weighing around a ton, the eland is the largest species of this animal in Africa",
  "category": "ANIMALS"
} 

ID: f1163de8-4035-46e0-9f5b-6304c3dcaa50
Data: {
  "answer": "the nose or snout",
  "question": "The gavial looks very much like a crocodile except for this bodily feature",
  "category": "ANIMALS"
} 

